# What is this notebook about?
This notebookstands for testing and demostrating pipeline of BPMN ai agent creation

## simple check if everything working fine

In [1]:
from openai import OpenAI
from settings import get_settings # set up scipt before testing

settings = get_settings()


AI_API_KEY = settings.OPENROUTER_API_KEY
MODEL_NAME = settings.OPENROUTER_MODEL_NAME

client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=AI_API_KEY,
)

completion = client.chat.completions.create(
    extra_headers={
        "HTTP-Referer": "<YOUR_SITE_URL>",  # Optional. Site URL for rankings on openrouter.ai.
        "X-Title": "<YOUR_SITE_NAME>",  # Optional. Site title for rankings on openrouter.ai.
    },
    extra_body={

        "reasoning": {

            "effort": "none" # Resoning turned off by note: https://openrouter.ai/xiaomi/mimo-v2-flash:free

        }

    },
    model=MODEL_NAME,
    messages=[{"role": "user", "content": "What is the meaning of life?"}],
    stream=True,
)
full_response = ""
for chunk in completion:
    if chunk.choices[0].delta.content:
        full_response += chunk.choices[0].delta.content
        print(chunk.choices[0].delta.content, end="", flush=True)


The meaning of life is one of the most profound and debated questions in philosophy, religion, science, and personal reflection. There is no single, universally accepted answer, but here are some perspectives from different fields:

### **1. Philosophical Perspectives**
- **Existentialism (Sartre, Camus, Nietzsche):** Life has no inherent meaning—it’s up to each individual to create their own purpose through choices, actions, and passions.
- **Absurdism (Camus):** Life may lack intrinsic meaning, but we must embrace the absurd and find joy in the struggle itself.
- **Stoicism:** Meaning comes from living virtuously, accepting what we cannot control, and focusing on reason and self-improvement.
- **Utilitarianism:** The meaning of life is to maximize happiness and reduce suffering for the greatest number of beings.

### **2. Religious & Spiritual Views**
- **Christianity/Judaism/Islam:** Life’s purpose is to serve, love, and worship God, following divine will and preparing for an afterl

# Graph pipeline
Lets start the experiments with pipline

In [1]:
import json
from settings import get_settings
from openai import OpenAI
from src.ai_generation.llm_client import LLMClient

settings = get_settings()


AI_API_KEY = settings.OPENROUTER_API_KEY
MODEL_NAME = settings.OPENROUTER_MODEL_NAME

raw_client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=AI_API_KEY
)

client = LLMClient(raw_client,MODEL_NAME, "Ты полезный ассистент")

full_response = ""

## 1. Schema planning

firstly generate needed schema for the task

In [2]:
with open(r'data\bpmn_schemas\plan.json', 'r', encoding='utf-8') as file:
    json_schema = json.load(file)

base_promt = "Write a BPMN schema for the task: Logistical company is planning to create a new product"
response = client.generate_response_json_based(prompt=base_promt, json_schema=json_schema)
print(json.dumps(json.loads(response), indent=4))

full_response += response

{
    "schema_name": "plan",
    "intent": "bpmn_diagram",
    "description": "BPMN diagram for a logistical company planning to create a new product.",
    "options": {
        "collaboration": true,
        "autoLayout": true,
        "includeDI": true
    }
}


## 2. Indetify nodes (processes) for task

In [3]:
with open(r'data\bpmn_schemas\process.json', 'r', encoding='utf-8') as file:
    json_schema = json.load(file)

response_processes = client.generate_response_json_based(prompt=full_response, json_schema=json_schema)
print(json.dumps(json.loads(response_processes), indent=4))

full_response += response_processes

{
    "schema_name": "process",
    "id": "new_product_plan",
    "name": "New Product Planning Process",
    "nodes": [
        {
            "id": "start",
            "name": "Start",
            "type": "startEvent"
        },
        {
            "id": "market_research",
            "name": "Conduct Market Research",
            "type": "task"
        },
        {
            "id": "feasibility_check",
            "name": "Check Feasibility",
            "type": "exclusiveGateway"
        },
        {
            "id": "feasible",
            "name": "Feasible",
            "type": "userTask"
        },
        {
            "id": "not_feasible",
            "name": "Not Feasible",
            "type": "endEvent"
        },
        {
            "id": "design_product",
            "name": "Design Product",
            "type": "task"
        },
        {
            "id": "prototype",
            "name": "Create Prototype",
            "type": "task"
        },
        {
          

## 3. Indetify flows
Create flows for the task nodes

In [4]:
with open(r'data\bpmn_schemas\flow.json', 'r', encoding='utf-8') as file:
    json_schema = json.load(file)

response_flows = client.generate_response_json_based(prompt=full_response, json_schema=json_schema)
print(json.dumps(json.loads(response_flows), indent=4))

full_response += response_flows

{
    "schema_name": "flows",
    "flows": [
        {
            "id": "flow1",
            "sourceRef": "start",
            "targetRef": "market_research"
        },
        {
            "id": "flow2",
            "sourceRef": "market_research",
            "targetRef": "feasibility_check"
        },
        {
            "id": "flow3",
            "name": "Feasible",
            "sourceRef": "feasibility_check",
            "targetRef": "feasible"
        },
        {
            "id": "flow4",
            "name": "Not Feasible",
            "sourceRef": "feasibility_check",
            "targetRef": "not_feasible"
        },
        {
            "id": "flow5",
            "sourceRef": "feasible",
            "targetRef": "design_product"
        },
        {
            "id": "flow6",
            "sourceRef": "design_product",
            "targetRef": "prototype"
        },
        {
            "id": "flow7",
            "sourceRef": "prototype",
            "targetRef": "test_

## 4. (OPTIONAL) Add collaboration
If there are some participants in the task, add collaboration

In [5]:
with open(r'data\bpmn_schemas\collaboration.json', 'r', encoding='utf-8') as file:
    json_schema = json.load(file)
    
response_collaboration = client.generate_response_json_based(prompt=full_response, json_schema=json_schema)
print(json.dumps(json.loads(response_collaboration), indent=4))

full_response += response_collaboration

{
    "schema_name": "collaboration",
    "id": "new_product_plan_collaboration",
    "participants": [
        {
            "id": "logistics_company",
            "name": "Logistics Company",
            "processRef": "new_product_plan"
        },
        {
            "id": "market_research_team",
            "name": "Market Research Team",
            "processRef": "market_research"
        },
        {
            "id": "design_team",
            "name": "Design Team",
            "processRef": "design_product"
        },
        {
            "id": "production_team",
            "name": "Production Team",
            "processRef": "production"
        }
    ]
}


## 5. (OPTIONAL) diagramm rendering / Logical DI
If there are some logical diagrams, add them to the task

In [6]:
with open(r'data\bpmn_schemas\diagramm_schema.json', 'r', encoding='utf-8') as file:
    json_schema = json.load(file)

response_di = client.generate_response_json_based(prompt=full_response, json_schema=json_schema)
print(json.dumps(json.loads(response_di), indent=4))

full_response += response_di

{
    "schema_name": "diagramm",
    "elements": [
        {
            "id": "start",
            "bpmnElement": "bpmn:StartEvent",
            "kind": "shape"
        },
        {
            "id": "market_research",
            "bpmnElement": "bpmn:Task",
            "kind": "shape"
        },
        {
            "id": "feasibility_check",
            "bpmnElement": "bpmn:ExclusiveGateway",
            "kind": "shape"
        },
        {
            "id": "feasible",
            "bpmnElement": "bpmn:UserTask",
            "kind": "shape"
        },
        {
            "id": "not_feasible",
            "bpmnElement": "bpmn:EndEvent",
            "kind": "shape"
        },
        {
            "id": "design_product",
            "bpmnElement": "bpmn:Task",
            "kind": "shape"
        },
        {
            "id": "prototype",
            "bpmnElement": "bpmn:Task",
            "kind": "shape"
        },
        {
            "id": "test_prototype",
            "bpmnEle

## 6. Layout

Can be migrated into script

In [7]:
with open(r'data\bpmn_schemas\layout.json', 'r', encoding='utf-8') as file:
    json_schema = json.load(file)

response_layout = client.generate_response_json_based(prompt=full_response, json_schema=json_schema)
print(json.dumps(json.loads(response_layout), indent=4))

full_response += response_layout

{
    "schema_name": "layout",
    "positions": [
        {
            "elementId": "start",
            "bounds": {
                "x": 50,
                "y": 50,
                "width": 36,
                "height": 36
            }
        },
        {
            "elementId": "market_research",
            "bounds": {
                "x": 50,
                "y": 120,
                "width": 100,
                "height": 80
            }
        },
        {
            "elementId": "feasibility_check",
            "bounds": {
                "x": 50,
                "y": 230,
                "width": 50,
                "height": 50
            }
        },
        {
            "elementId": "feasible",
            "bounds": {
                "x": 150,
                "y": 210,
                "width": 100,
                "height": 80
            }
        },
        {
            "elementId": "not_feasible",
            "bounds": {
                "x": 150,
              

# 8. (Optional) Validation

Add sooner

# BPMN generating

## Assembling data

### JSON

In [8]:
from src.assemblers.json.bpmn import BpmnJsonAssembler

assembler = BpmnJsonAssembler()
final_json = assembler.add_parts_from_list([json.loads(response_processes), json.loads(response_flows), json.loads(response_collaboration), json.loads(response_di), json.loads(response_layout)])
assembler.save_to_file(r"final_schema.json")

True

In [9]:
normilized_dict = assembler.get_data_for_director()

### XML

In [10]:
from src.assemblers.xml.director import BpmnDirector
from src.assemblers.xml.bpmn import BpmnBuilder

xml_assembler = BpmnDirector(BpmnBuilder())
xml_assembler.construct_from_json(normilized_dict)
xml_assembler.save_to_file(r"temp_data\Draft.xml")

True

# GRAPH testing

In [ ]:
from src.ai_generation.bpmn_agent.agent import get_agent

agent = get_agent()

initial_state = {
    "messages":["Create BPMN schema for the task: Logistical company is planning to create a new product."]
}
result = agent.invoke(initial_state)

print(result["messages"][-1])

TypeError: can only concatenate list (not "str") to list